# Preliminaries

In [522]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [523]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

warnings.filterwarnings('ignore')

Import the data

In [524]:
path = "../Datasets/"
Train = pd.read_csv(path + '/Historical-transaction-data.csv')
StoreInfo = pd.read_csv(path + '/Store-info.csv')
Submission_testing = pd.read_csv(path + '/Testing-data.csv')

Join both datasets to form a train dataset

In [525]:
Train = Train.merge(StoreInfo, on='shop_id', how='left')

In [526]:
Submission_testing.drop("shop_profile", axis=1, inplace=True)
# Submission_testing = Submission_testing.merge(StoreInfo, on='shop_id', how='left')
# Submission_testing.drop("shop_profile", axis=1, inplace=True)
Submission_testing = Submission_testing.merge(Train, on='shop_id', how='left')

In [527]:
Submission_testing.drop("shop_profile", axis=1, inplace=True)

In [528]:
# Train.head()

In [529]:
# Submission_testing.head()

In [530]:
print("New length: ", Train.shape[0])

New length:  473974


Remove Duplicates

In [531]:
i_length = Train.shape[0]

In [532]:
Train.drop_duplicates(inplace=True)

In [533]:

print("Duplicates dropped from Train: ", i_length - Train.shape[0] )
print("New length: ", Train.shape[0])

Duplicates dropped from Train:  31211
New length:  442763


# Analysis

Missing data

In [534]:
# plt.figure(figsize=(10,8))
# cols = Train.columns
# colours = ['#000099', '#ffff00'] # specify colours: yellow - missing. blue - not missing
# sns.heatmap(Train[cols].isna(), cmap=sns.color_palette(colours))

In [535]:
# # No. of columns missing for the rows
# missing_by_row = Train.isna().sum(axis="columns")
# missing_by_row.hist(bins=100)

In [536]:
pct_missing = Train.isna().mean()
pct_missing[:15]

item_description    0.079198
transaction_date    0.000000
invoice_id          0.013030
customer_id         0.000000
shop_id             0.000000
item_price          0.000000
quantity_sold       0.000000
shop_area_sq_ft     0.000000
shop_profile        0.183565
dtype: float64

In [537]:
#Remove rows with null values for item_description or shop_profile
Train = Train.dropna(subset=['item_description', 'shop_profile'], axis=0)

In [538]:
# Train.info()

In [539]:
print("New length: ", Train.shape[0])

New length:  332581


Fill Missing Values

In [540]:
# Train.info()

Encoding

In [541]:
# Set the data types of each feature

# The nominative (unordered) categorical features
features_nom = ["item_description", "customer_id", "shop_id", "shop_profile"]

for feature in features_nom:
    Train[feature] = Train[feature].astype("category")
    if feature == "shop_profile":
        continue
    Submission_testing[feature] = Submission_testing[feature].astype("category")

Train["transaction_date"] = pd.to_datetime(Train["transaction_date"])

In [542]:
Train_y = Train["shop_profile"]
Train.drop("shop_profile", axis=1, inplace=True)

In [543]:
# Train.info()

In [544]:
Train.drop(["invoice_id", "customer_id"], axis=1, inplace=True)
Submission_testing.drop(["invoice_id", "customer_id"], axis=1, inplace=True)

In [545]:
# # creating mask
# mask = np.triu(np.ones_like(Train.corr()))
 
# # plotting a triangle correlation heatmap
# dataplot = sns.heatmap(Train.corr(), cmap="YlGnBu", annot=True, mask=mask)
 
# # displaying heatmap
# plt.show()

In [546]:
# Check for outliers

Train.kurt(numeric_only=True)

item_price         382.227475
quantity_sold      420.507197
shop_area_sq_ft      2.908254
dtype: float64

In [547]:
Train["quantity_sold"].describe()

count    332581.000000
mean          1.874626
std           1.539118
min          -1.000000
25%           1.000000
50%           2.000000
75%           2.000000
max         101.000000
Name: quantity_sold, dtype: float64

In [548]:
# TODO : further investigate the outliers

In [549]:
Train["item_price"].describe()

count    332581.000000
mean        214.825306
std         167.916289
min          35.000000
25%         110.000000
50%         220.000000
75%         220.000000
max       17400.000000
Name: item_price, dtype: float64

In [550]:
Train['timestamp_dt'] = pd.to_datetime(Train['transaction_date'], format='%Y-%m-%d')
Train['year'] = Train['timestamp_dt'].dt.year
Train['month'] = Train['timestamp_dt'].dt.month
Train['weekday'] = Train['timestamp_dt'].dt.weekday

Train.drop(['timestamp_dt', 'transaction_date'], axis=1, inplace=True)

Train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,ORANGE BARLEY 1.5L,SHOP008,220,2,678,2021,12,5
1,GINGER BEER 1.5L,SHOP112,220,2,668,2021,10,6
2,TONIC PET 500ML,SHOP008,160,2,678,2021,12,0
3,CREAM SODA 1L,SHOP008,150,2,678,2021,12,0
4,STRAWBERRY MILK 180ML,SHOP112,210,5,668,2021,10,5


In [551]:
Submission_testing['timestamp_dt'] = pd.to_datetime(Submission_testing['transaction_date'], format='%Y-%m-%d')
Submission_testing['year'] = Submission_testing['timestamp_dt'].dt.year
Submission_testing['month'] = Submission_testing['timestamp_dt'].dt.month
Submission_testing['weekday'] = Submission_testing['timestamp_dt'].dt.weekday

Submission_testing.drop(['timestamp_dt', 'transaction_date'], axis=1, inplace=True)

Submission_testing.head()

,shop_id,item_description,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,SHOP046,FIT O MIXED FRUIT 1L,270,1,545,2021,10,4
1,SHOP046,FIT O ORANGE 1L,290,1,545,2021,10,2
2,SHOP046,LEMONADE 1.5L,220,2,545,2021,10,3
3,SHOP046,FIT O MANGO 200ML,180,4,545,2021,11,2
4,SHOP046,FIT O MIXED FRUIT 200ML,60,1,545,2021,11,2


In [552]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Train_y = le.fit_transform(Train_y)

In [553]:
# remove the substring "SHOP" from the shop_id column and convert it to int
Train["shop_id"] = Train["shop_id"].str.replace("SHOP", "").astype(int)
# Train["shop_id"] = Train["shop_id"].str.replace("SHOP", "")
# Train["shop_id"] = Train["shop_id"].astype()")
# convert it to categorical
Train["shop_id"] = Train["shop_id"].astype("category")

In [554]:
# remove the substring "SHOP" from the shop_id column
Submission_testing["shop_id"] = Submission_testing["shop_id"].str.replace("SHOP", "").astype(int)

In [555]:
## Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Train, Train_y, test_size = 0.2, random_state = 1)

In [556]:
X_train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
13201,SODA 1L,86,110,2,661,2021,10,5
293065,LEMONADE 500ML,37,200,2,717,2021,12,5
63548,LEMONADE 1.5L,66,220,3,470,2021,12,0
466484,GINGER BEER 1.5L,60,220,1,676,2021,10,2
316348,SODA 1L,112,220,3,668,2021,11,0


In [557]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [558]:
# X_train.info()

In [559]:
Submission_testing = Submission_testing[Train.columns]

In [560]:
# X_train.head()

In [561]:
# unique values in each column
X_train.nunique()

item_description     37
shop_id             100
item_price          172
quantity_sold        59
shop_area_sq_ft      53
year                  1
month                 3
weekday               7
dtype: int64

In [562]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266064 entries, 13201 to 182378
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   item_description  266064 non-null  category
 1   shop_id           266064 non-null  category
 2   item_price        266064 non-null  int64   
 3   quantity_sold     266064 non-null  int64   
 4   shop_area_sq_ft   266064 non-null  int64   
 5   year              266064 non-null  int64   
 6   month             266064 non-null  int64   
 7   weekday           266064 non-null  int64   
dtypes: category(2), int64(6)
memory usage: 14.7 MB


In [563]:
categorical_columns = ["year", "month", "weekday"]

for category in categorical_columns:
    Train[category] = Train[category].astype("category")

# Metrics

In [564]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


def score_classification(model, df1_x, df1_y, df2_x, df2_y):
  print(model)
  df1_x = df1_x.copy()
  df2_x = df2_x.copy()
  
  ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0])], remainder='passthrough')
 
  # One hot encoding for the item_description column

  df1_encoded = ct.fit_transform(df1_x[["item_description"]])
  df1_x = pd.concat([df1_x.drop("item_description", axis=1).reset_index(drop=True), pd.DataFrame(df1_encoded.toarray())], axis='columns')
 
  df2_encoded = ct.transform(df2_x[["item_description"]]) 
  df2_x = pd.concat([df2_x.drop("item_description", axis=1).reset_index(drop=True), pd.DataFrame(df2_encoded.toarray())], axis='columns')    
  
  df1_x.columns = df1_x.columns.astype(str)
  df2_x.columns = df2_x.columns.astype(str)
 
  # Fit the model  
  model.fit(df1_x, df1_y)

  # Make predictions
  y_pred = model.predict(df2_x)

  # Evaluate the model
  accuracy = accuracy_score(df2_y, y_pred)
  precision = precision_score(df2_y, y_pred,  average='macro')
  recall = recall_score(df2_y, y_pred,  average='macro')
  f1 = f1_score(df2_y, y_pred,  average='macro')

  print(f"Accuracy: {accuracy}")
  print(f"Precision: {precision}")
  print(f"Recall: {recall}")
  print(f"F1 Score: {f1}")

In [565]:
Submission_testing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86633 entries, 0 to 86632
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   item_description  80319 non-null  category
 1   shop_id           86633 non-null  int32   
 2   item_price        86633 non-null  int64   
 3   quantity_sold     86633 non-null  int64   
 4   shop_area_sq_ft   86633 non-null  int64   
 5   year              86633 non-null  int64   
 6   month             86633 non-null  int64   
 7   weekday           86633 non-null  int64   
dtypes: category(1), int32(1), int64(6)
memory usage: 5.0 MB


## Result Output

In [620]:
def outputResult(model, df1_x, df1_y, test, le):
    print(model)
    df1_x = df1_x.copy()
    df2_x = test.copy()
    
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0])], remainder='passthrough')

    # One hot encoding for the item_description column

    df1_encoded = ct.fit_transform(df1_x[["item_description"]])
    df1_x = pd.concat([df1_x.drop("item_description", axis=1).reset_index(drop=True), pd.DataFrame(df1_encoded.toarray())], axis='columns')

    df2_encoded = ct.transform(df2_x[["item_description"]])
    df2_x = pd.concat([df2_x.drop("item_description", axis=1).reset_index(drop=True), pd.DataFrame(df2_encoded.toarray())], axis='columns')

    df1_x.columns = df1_x.columns.astype(str)
    df2_x.columns = df2_x.columns.astype(str)

    # Fit the model on the training data
    model.fit(df1_x, df1_y)
    
    # Make predictions on the test data
    y_pred = model.predict(df2_x)
    
    # Inverse transform the encoded predictions to the original shop profiles
    y_pred = le.inverse_transform(y_pred)    
    
    # Create a dataframe with the predicted shop profiles
    results_df = pd.DataFrame({'shop_id': test.shop_id, 'shop_profile': y_pred})    
    # rename the shop_id column with the word "SHOP" and add a 0 in front of the shop_id
    results_df["shop_id"] = results_df["shop_id"].astype(str)
    results_df["shop_id"] = "SHOP" + results_df["shop_id"].str.zfill(3)
    
    # group the dataframe by shop_id and use the most frequent shop_profile
    results_df = results_df.groupby('shop_id', sort=False)['shop_profile'].agg(lambda x:x.value_counts().index[0]).reset_index()

    results_df.drop_duplicates(inplace=True)
    # Write the dataframe to a CSV file
    results_df.to_csv('predictions'+str(model).strip("()")+'.csv', index=False)

# Modelling

In [567]:
# unique values in item_description column

print(X_train['item_description'].unique())

# no of null values in item_description column

print(X_train['item_description'].isnull().sum())

['SODA 1L', 'LEMONADE 500ML', 'LEMONADE 1.5L', 'GINGER BEER 1.5L', 'SODA 500ML', ..., 'GINGER BEER SUGAR FREE 500ML', 'LEMONADE 1L', 'NECTO 1L', 'CHOCOLATE MILK 180ML', 'LIME CRUSH JUICE 1L']
Length: 37
Categories (37, object): ['BOTTLED DRINKING WATER 1.5L', 'BOTTLED DRINKING WATER 500ML', 'CHOCOLATE MILK 180ML', 'CREAM SODA 1.5L', ..., 'SODA PET 1.5L', 'STRAWBERRY MILK 180ML', 'TONIC PET 500ML', 'TWISTEE APPLE 1L']
0


In [568]:
from sklearn.linear_model import LogisticRegression
import sys
logr = LogisticRegression()
# np.set_printoptions(threshold=np.inf)
score_classification(logr, X_train, y_train, X_test, y_test)
# np.set_printoptions(threshold=1000)

LogisticRegression()
Accuracy: 0.42414721048754456
Precision: 0.41393245648700505
Recall: 0.4099652958364213
F1 Score: 0.4041638373147771


In [569]:
Submission_testing.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,FIT O MIXED FRUIT 1L,46,270,1,545,2021,10,4
1,FIT O ORANGE 1L,46,290,1,545,2021,10,2
2,LEMONADE 1.5L,46,220,2,545,2021,10,3
3,FIT O MANGO 200ML,46,180,4,545,2021,11,2
4,FIT O MIXED FRUIT 200ML,46,60,1,545,2021,11,2


In [621]:
outputResult(logr, X_train, y_train, Submission_testing, le)

LogisticRegression()


Bug with KNeighborsClassifiers

In [ ]:
#Dont use this
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

score_classification(knn, X_train, y_train, X_test, y_test)

In [623]:
outputResult(knn, X_train, y_train, Submission_testing, le)

KNeighborsClassifier()


In [617]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

score_classification(dtc, X_train, y_train, X_test, y_test)

DecisionTreeClassifier()
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [574]:
X_train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
13201,SODA 1L,86,110,2,661,2021,10,5
293065,LEMONADE 500ML,37,200,2,717,2021,12,5
63548,LEMONADE 1.5L,66,220,3,470,2021,12,0
466484,GINGER BEER 1.5L,60,220,1,676,2021,10,2
316348,SODA 1L,112,220,3,668,2021,11,0


In [575]:
Submission_testing.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,FIT O MIXED FRUIT 1L,46,270,1,545,2021,10,4
1,FIT O ORANGE 1L,46,290,1,545,2021,10,2
2,LEMONADE 1.5L,46,220,2,545,2021,10,3
3,FIT O MANGO 200ML,46,180,4,545,2021,11,2
4,FIT O MIXED FRUIT 200ML,46,60,1,545,2021,11,2


In [576]:
outputResult(dtc, X_train, y_train, Submission_testing, le)

DecisionTreeClassifier()


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

score_classification(rfc, X_train, y_train, X_test, y_test)

In [618]:
outputResult(rfc, X_train, y_train, Submission_testing, le)

RandomForestClassifier()


TypeError: could not convert string to float: 'Moderate'

In [579]:
# from sklearn.naive_bayes import GaussianNB

# nb = GaussianNB()

# score_classification(nb, X_train, y_train, X_test, y_test)

In [615]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100, max_depth=5)

score_classification(xgb_model, X_train, y_train, X_test, y_test)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:shop_id: category

# Feature Engineering

Mutual Information

In [581]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X,y):
  X = X.copy()
  for colname in X.select_dtypes(["object", "category"]):
    X[colname], _ = X[colname].factorize()
  # all discrete features should now have integer dtypes
  discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
  mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
  mi_scores = pd.Series(mi_scores, name="Mutual Information Scores", index=X.columns)
  mi_scores = mi_scores.sort_values(ascending=False)
  return mi_scores


def plot_mi_scores(scores):
  scores = scores.sort_values(ascending=True)
  width = np.arange(len(scores))
  ticks = list(scores.index)
  plt.barh(width,scores)
  plt.yticks(width, ticks)
  plt.title("Mututal Information Scores")

In [582]:
# mi_scores = make_mi_scores(X_train, y_train)
# mi_scores

KeyboardInterrupt: 

In [ ]:
X_train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
13201,SODA 1L,86,110,2,661,2021,10,5
293065,LEMONADE 500ML,37,200,2,717,2021,12,5
63548,LEMONADE 1.5L,66,220,3,470,2021,12,0
466484,GINGER BEER 1.5L,60,220,1,676,2021,10,2
316348,SODA 1L,112,220,3,668,2021,11,0


Feature Scaling

In [ ]:
X_train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
13201,SODA 1L,86,110,2,661,2021,10,5
293065,LEMONADE 500ML,37,200,2,717,2021,12,5
63548,LEMONADE 1.5L,66,220,3,470,2021,12,0
466484,GINGER BEER 1.5L,60,220,1,676,2021,10,2
316348,SODA 1L,112,220,3,668,2021,11,0


In [583]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

ct2 = ColumnTransformer([("sScaler", StandardScaler(), []),
                         ("MinMaxScaler", MinMaxScaler(), ["shop_area_sq_ft","item_price", "quantity_sold"])])
scaled_df1_x = pd.DataFrame(ct2.fit_transform(X_train), columns=["shop_area_sq_ft", "item_price", "quantity_sold"])
scaled_df2_x = pd.DataFrame(ct2.transform(X_test), columns=["shop_area_sq_ft", "item_price", "quantity_sold"])
scaled_submission_df_x = pd.DataFrame(ct2.transform(Submission_testing), columns=["shop_area_sq_ft", "item_price", "quantity_sold"])

# Create copies of X_train and X_test with the scaled values
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_submission_scaled = Submission_testing.copy()


# Assign the scaled values to the new dataframes
X_train_scaled[scaled_df1_x.columns] = scaled_df1_x
X_test_scaled[scaled_df2_x.columns] = scaled_df2_x
X_submission_scaled[scaled_submission_df_x.columns] = scaled_submission_df_x

In [ ]:
X_train.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,SODA 1L,86,110,2,661,2021,10,5
1,LEMONADE 500ML,37,200,2,717,2021,12,5
2,LEMONADE 1.5L,66,220,3,470,2021,12,0
3,GINGER BEER 1.5L,60,220,1,676,2021,10,2
4,SODA 1L,112,220,3,668,2021,11,0


In [ ]:
X_test_scaled.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,GINGER BEER 1L,10,0.006623,0.019608,0.409499,2021,12,4
1,SODA 500ML,127,0.002016,0.019608,0.706033,2021,10,1
2,GINGER BEER 1.5L,84,0.010654,0.019608,0.409499,2021,10,1
3,GINGER BEER 1.5L,17,0.010654,0.019608,0.409499,2021,12,1
4,SODA 1L,83,0.004319,0.029412,0.485237,2021,12,1


In [584]:

print("Scores after standard scaling: ")
score_classification(rfc, X_train_scaled, y_train, X_test_scaled, y_test)

Scores after standard scaling: 
RandomForestClassifier()
Accuracy: 0.9752995474841019
Precision: 0.9754218704498799
Recall: 0.9746671944958739
F1 Score: 0.9750152235636746


In [585]:
score_classification(dtc, X_train_scaled, y_train, X_test_scaled, y_test)

DecisionTreeClassifier()
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [586]:
from sklearn.decomposition import PCA

def apply_pca(X):
  pca= PCA()
  df1_x = X.copy()
  ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0])], remainder='passthrough')

  # One hot encoding for the item_description column

  df1_encoded = ct.fit_transform(df1_x[["item_description"]])
  df1_x = pd.concat([df1_x.drop("item_description", axis=1).reset_index(drop=True), pd.DataFrame(df1_encoded.toarray())], axis='columns')

  df1_x.columns = df1_x.columns.astype(str)

  X_pca = pca.fit_transform(df1_x)
  component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
  X_pca = pd.DataFrame(X_pca, columns=component_names)
  #create loadings
  loadings = pd.DataFrame(
  pca.components_.T,
  columns = component_names,
  index=df1_x.columns,
  )
  return pca, X_pca, loadings

def plot_variance(pca, width=8, dpi=100):
  fig, axs = plt.subplots(1,2)
  n = pca.n_components_
  grid = np.arange(1, n+1)
  evr = pca.explained_variance_ratio_
  axs[0].bar(grid,evr)
  axs[0].set(
      xlabel="Component",title="% Explained Variance", ylim=(0.0,1.0)             )
  #Cumulative Variance
  cv = np.cumsum(evr)
  axs[1].plot(np.r_[0,grid], np.r_[0,cv], "o-")
  axs[1].set(xlabel="Component", title="% Cumulatve Variance", ylim=(0.0, 1.0))
  fig.set(figwidth=8, dpi=100)
  return axs

In [ ]:
# pca, X_pca, loadings = apply_pca(X_train_scaled)
# print(loadings)

                          PC1           PC2           PC3           PC4  \
shop_id          9.999999e-01  4.116362e-04  2.930904e-05  6.532546e-05   
item_price       2.802056e-06  5.847241e-07 -4.588511e-04  2.174064e-03   
quantity_sold    4.081212e-06  6.545116e-05 -6.837778e-05 -1.860780e-03   
shop_area_sq_ft -1.236992e-04 -6.843135e-04  1.459271e-03 -3.732076e-03   
year            -1.577722e-30 -3.252607e-19 -5.551115e-17 -1.110223e-16   
month            3.023357e-05  1.074471e-02 -9.927798e-01 -6.496051e-02   
weekday          4.123347e-04 -9.999343e-01 -1.037887e-02 -2.114663e-03   
0                3.538843e-05  4.068883e-04 -3.315671e-04 -1.910297e-02   
1                4.679017e-05  7.441194e-04 -5.964619e-04 -4.126000e-02   
2               -3.081984e-07  6.218528e-06 -1.296671e-04 -9.884733e-05   
3               -2.438986e-05 -5.020817e-04 -1.177097e-02 -2.556760e-01   
4               -9.818263e-06 -8.022412e-05  1.268312e-04 -7.160366e-03   
5               -4.283542

In [ ]:
# mi_scores = make_mi_scores(X_pca, y_train)
# mi_scores

PC1     1.092137
PC3     0.922312
PC2     0.813435
PC4     0.705458
PC20    0.526378
PC5     0.520097
PC10    0.517199
PC8     0.503522
PC18    0.498645
PC23    0.492386
PC14    0.489111
PC28    0.488910
PC33    0.484306
PC22    0.483800
PC17    0.480447
PC13    0.477267
PC35    0.476823
PC19    0.476669
PC15    0.475993
PC24    0.475980
PC42    0.475768
PC6     0.472890
PC36    0.472135
PC7     0.471602
PC21    0.470641
PC11    0.468679
PC16    0.467102
PC9     0.466920
PC31    0.466909
PC29    0.465286
PC30    0.464821
PC12    0.464109
PC37    0.463963
PC38    0.462273
PC26    0.460995
PC39    0.460308
PC41    0.458861
PC40    0.458274
PC34    0.457402
PC27    0.455995
PC32    0.455208
PC25    0.454391
PC44    0.448442
PC43    0.448416
Name: Mutual Information Scores, dtype: float64

In [ ]:
X_train_scaled.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,SODA 1L,86,0.004319,0.029412,0.465982,2021,10,5
1,LEMONADE 500ML,37,0.009502,0.029412,0.537869,2021,12,5
2,LEMONADE 1.5L,66,0.010654,0.039216,0.220796,2021,12,0
3,GINGER BEER 1.5L,60,0.010654,0.019608,0.485237,2021,10,2
4,SODA 1L,112,0.010654,0.039216,0.474968,2021,11,0


In [587]:
# Create price-related features
X_train_scaled['total_sales'] = X_train_scaled['item_price'] * X_train_scaled['quantity_sold']
# Create shop-related features
# X_train_scaled['shop_sales_per_sq_ft'] = X_train_scaled['total_sales'] / X_train_scaled['shop_area_sq_ft']
X_train_scaled['shop_sales_rank'] = X_train_scaled.groupby('shop_id')['total_sales'].rank(ascending=False)


X_test_scaled['total_sales'] = X_test_scaled['item_price'] * X_test_scaled['quantity_sold']
# X_test_scaled['shop_sales_per_sq_ft'] = X_test_scaled['total_sales'] / X_test_scaled['shop_area_sq_ft']
X_test_scaled['shop_sales_rank'] = X_test_scaled.groupby('shop_id')['total_sales'].rank(ascending=False)

X_submission_scaled['total_sales'] = X_submission_scaled['item_price'] * X_submission_scaled['quantity_sold']
# X_submission_scaled['shop_sales_per_sq_ft'] = X_submission_scaled['total_sales'] / X_submission_scaled['shop_area_sq_ft']
X_submission_scaled['shop_sales_rank'] = X_submission_scaled.groupby('shop_id')['total_sales'].rank(ascending=False)

In [588]:
# remove the redundant columns

X_train_scaled.drop(['item_price', 'quantity_sold'], axis=1, inplace=True)
X_test_scaled.drop(['item_price', 'quantity_sold'], axis=1, inplace=True)
X_submission_scaled.drop(['item_price', 'quantity_sold'], axis=1, inplace=True)

In [589]:
X_train_scaled.head()

,item_description,shop_id,shop_area_sq_ft,year,month,weekday,total_sales,shop_sales_rank
0,SODA 1L,86,0.465982,2021,10,5,0.000127,3003.5
1,LEMONADE 500ML,37,0.537869,2021,12,5,0.000279,827.5
2,LEMONADE 1.5L,66,0.220796,2021,12,0,0.000418,580.0
3,GINGER BEER 1.5L,60,0.485237,2021,10,2,0.000209,1127.5
4,SODA 1L,112,0.474968,2021,11,0,0.000418,285.0


In [590]:
X_train_scaled.describe()

,shop_area_sq_ft,year,month,weekday,total_sales,shop_sales_rank
count,266064.000000,266064.0,266064.000000,266064.000000,266064.000000,266064.000000
mean,0.430169,2021.0,11.024182,3.341437,0.000379,1449.060155
std,0.159823,0.0,0.776629,2.012698,0.001923,940.751166
min,0.000000,2021.0,10.000000,0.000000,0.000000,1.000000
25%,0.394095,2021.0,10.000000,2.000000,0.000110,694.500000
50%,0.409499,2021.0,11.000000,4.000000,0.000209,1341.000000
75%,0.485237,2021.0,12.000000,5.000000,0.000313,2091.500000
max,1.000000,2021.0,12.000000,6.000000,0.607843,4513.000000


In [591]:
#check for missing values

X_train_scaled.isnull().sum()

item_description    0
shop_id             0
shop_area_sq_ft     0
year                0
month               0
weekday             0
total_sales         0
shop_sales_rank     0
dtype: int64

In [593]:
X_submission_scaled.isnull().sum()

item_description    6314
shop_id                0
shop_area_sq_ft        0
year                   0
month                  0
weekday                0
total_sales            0
shop_sales_rank        0
dtype: int64

In [594]:
# drop the rows with missing values

X_train_scaled.dropna(inplace=True)
X_test_scaled.dropna(inplace=True)
X_submission_scaled.dropna(inplace=True)

In [ ]:
# # Impute missing values for the shop_sales_per_sq_ft column

# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(missing_values = np.nan, strategy="mean")
# imputer.fit(X_train_scaled[["shop_sales_per_sq_ft"]])

# X_train_scaled = pd.DataFrame(imputer.transform(X_train_scaled), columns=["shop_sales_per_sq_ft"]])
# X_test_scaled = pd.DataFrame(imputer.transform(X_test_scaled), columns=X_train_scaled.columns)
# X_submission_scaled = pd.DataFrame(imputer.transform(X_submission_scaled), columns=X_submission_scaled.columns)

In [626]:
score_classification(dtc, X_train_scaled, y_train, X_test_scaled, y_test)

DecisionTreeClassifier()
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [628]:
outputResult(dtc, X_train_scaled, y_train, X_submission_scaled, le)

DecisionTreeClassifier()


In [629]:
score_classification(rfc, X_train_scaled, y_train, X_test_scaled, y_test)

RandomForestClassifier()
Accuracy: 0.7716523595471835
Precision: 0.8076157953943078
Recall: 0.7880105700882468
F1 Score: 0.7681265650899315


In [632]:
outputResult(rfc, X_train_scaled, y_train, X_submission_scaled, le)

RandomForestClassifier()


In [633]:
score_classification(knn, X_train_scaled, y_train, X_test_scaled, y_test)

KNeighborsClassifier()
Accuracy: 0.5093134086023122
Precision: 0.5081336157350509
Recall: 0.5097537277825253
F1 Score: 0.5085792450709201


In [634]:
outputResult(knn, X_train_scaled, y_train, X_submission_scaled, le)

KNeighborsClassifier()


In [597]:
Submission_testing.head()

,item_description,shop_id,item_price,quantity_sold,shop_area_sq_ft,year,month,weekday
0,FIT O MIXED FRUIT 1L,46,270,1,545,2021,10,4
1,FIT O ORANGE 1L,46,290,1,545,2021,10,2
2,LEMONADE 1.5L,46,220,2,545,2021,10,3
3,FIT O MANGO 200ML,46,180,4,545,2021,11,2
4,FIT O MIXED FRUIT 200ML,46,60,1,545,2021,11,2
